# Tensorflow intro

This tutorial shows the basic usage of tensorflow to train neural networks

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Simple NN for classification

## Data processing
Read data and convert them to numerical inputs
Vytvárame dáta: X(matica rozmerov počet pozorovaní krát dimenzionalita dát)->inputy, y(vektor, pre každé pozorovanie máme jednu hodnotu)-> outputy

In [3]:
X, y = make_classification(n_samples=20000, n_features=8, n_informative=5, 
                           n_redundant=0, n_classes=2, random_state=1)

In [4]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print('train size: {}, target_ratio: {:.3f}'.format(X_train.shape, np.mean(y_train)))
print('test size: {}, target_ratio: {:.3f}'.format(X_test.shape, np.mean(y_test)))
print('dev size: {}, target_ratio: {:.3f}'.format(X_dev.shape, np.mean(y_dev)))

train size: (12800, 8), target_ratio: 0.502
test size: (4000, 8), target_ratio: 0.501
dev size: (3200, 8), target_ratio: 0.502


## Building a simple model with tf.keras

Very useful documentations with many examples and detailed explanation of everything you might need:
 - https://www.tensorflow.org/api_docs/python/tf/keras/
 - https://keras.io/api/

Contain everything about:
  - Model building: Activations, Losses, Optimizers, Regularization
  - Data processing
  - Pretrained models and datasets
  - Automatic differentiation
  - ...

  

### Model speficication

three APIs for building the model
   - sequential - easy to code, but less flexible - we will use it sometimes
   - functional - flexible and still easy to code - we will use it the most
   - model subclassing - rather complicated and not very much used - we will skip it

#### Sequential API

(https://www.tensorflow.org/guide/keras/sequential_model)

Easy to code but <span style="color:red"> NOT </span> appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [9]:
# Specification A)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([X_train.shape[1],]), # Create input layer with 'input data' neurons
    tf.keras.layers.Dense(10, activation="relu"), # Create hidden layer with 10 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation="sigmoid"), # Create output layer with one neuron and sigmoid activation
])

model.summary()
#na vektor urobí výpočet a vyhodí jednu hodnotu
#v nákrese modela budeme mať maticu, ktorá reprezentuje prechod z vektora na 10 uzlov
#layer 0. matica 10x8 bias 10
# 11, lebo zacinam s 10 dimenzionalnym vektorom
#90 lebo 10x8 + 10 bias
#tu sú všetky parametre trainable

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                90        
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Specification B)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                90        
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


#### Functional API

(https://www.tensorflow.org/guide/keras/functional)

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.



In [11]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

hidden = tf.keras.layers.Dense(10)(inputs)#vlozim ho do hidden
hidden = tf.keras.activations.relu(hidden)
hidden = tf.keras.layers.Dense(1)(hidden)
outputs = tf.keras.activations.sigmoid(hidden)#toto mi z toho urobi jednodimenzionalny vystup

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Model')

#konstrukteru neposielam vsetky layers vopred, ale najprv vezmem inouts a dam ho ako argument dalsiemu layeru

In [15]:
model.summary()#konstruovali sme rovnaky model, parametry su dimenzionalitou rovnake ako predtym

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_8 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 10)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid (TFOpLambda  (None, 1)                0         
 )                                                               
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_______________________________________________________________

In [14]:
# TODO: To make the following line work you need to install graphviz (if you have not done so in one of the previous classes)
# 1) follow the instructions https://graphviz.gitlab.io/download/?fbclid=IwAR1V-lrRhho5rSfBVYXYISsighqRwOCOgMHLmL_DclkQrPtMXQaKj3mFcqs
# 2) this notebook has been tested with version 8.0.3
# 3) make sure you add it to the PATH variable (you are specifically asked during the installation) at least for local user

tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


### Model compilation and training

In [16]:
# compile the model with selected optimizer, loss and metrics
model.compile(
        optimizer=tf.optimizers.Adam(), # Several other possibilities for optimizers 
        loss=tf.losses.BinaryCrossentropy(), # Select the proper loss for the task
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()], # Select the proper metrics for the task
)

In [17]:
print('\n>>> Bias of the last layers:')
print(model.layers[3].weights[1].numpy())

print('\n>>> Kernel of the last layers:')
print(model.layers[3].weights[0].numpy())

print('\n>>> Bias of the first layers:')
print(model.layers[1].weights[1].numpy())

print('\n>>> Kernel of the first layers:')
print(model.layers[1].weights[0].numpy())#kernel značí maticu


>>> Bias of the last layers:
[0.]

>>> Kernel of the last layers:
[[-0.72893476]
 [-0.71156394]
 [-0.2425799 ]
 [-0.6511235 ]
 [ 0.37352353]
 [-0.68818015]
 [ 0.34182936]
 [ 0.00153458]
 [-0.3081322 ]
 [-0.35564047]]

>>> Bias of the first layers:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

>>> Kernel of the first layers:
[[-0.31383747  0.06594914 -0.29382563  0.5652292  -0.54854995 -0.54845333
   0.22560358 -0.30288085 -0.52501136 -0.4196092 ]
 [-0.44453394  0.20297885  0.28307647 -0.39552096 -0.36806387 -0.293498
   0.19550467  0.00085616 -0.4780521   0.04681826]
 [-0.46890563  0.21891373  0.54611063 -0.04934472 -0.13326505 -0.1936644
   0.28158736 -0.37767726  0.25715292 -0.0426743 ]
 [-0.09494659  0.35307693 -0.4111284   0.33889478  0.5138445   0.02470738
   0.45941424  0.10407823 -0.41524112  0.10758251]
 [ 0.42415273 -0.46851316 -0.24430388  0.4363283  -0.5718684   0.40568852
  -0.16792339  0.19183308  0.50850534 -0.45830676]
 [-0.12345082  0.45700085 -0.4183625   0.4960717   0.08358777 -0

In [18]:
# train the model with default setting
model.fit(X_train, y_train, batch_size=64, epochs=100)         

Epoch 1/100
200/200 [==============================] - 2s 2ms/step - loss: 0.7865 - auc: 0.6481 - binary_accuracy: 0.6016
Epoch 2/100
200/200 [==============================] - 0s 2ms/step - loss: 0.5029 - auc: 0.8516 - binary_accuracy: 0.7720
Epoch 3/100
200/200 [==============================] - 0s 2ms/step - loss: 0.4298 - auc: 0.8969 - binary_accuracy: 0.8131
Epoch 4/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3867 - auc: 0.9164 - binary_accuracy: 0.8436
Epoch 5/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3578 - auc: 0.9270 - binary_accuracy: 0.8602
Epoch 6/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3366 - auc: 0.9344 - binary_accuracy: 0.8723
Epoch 7/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3203 - auc: 0.9401 - binary_accuracy: 0.8797
Epoch 8/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3076 - auc: 0.9442 - binary_accuracy: 0.8831
Epoch 9/100
200/200 [===

In [19]:
# Evaluate the model and predict for the test data
model.evaluate(X_test, y_test)
test_pred = model.predict(X_test)

125/125 [==============================] - 1s 3ms/step - loss: 0.2265 - auc: 0.9693 - binary_accuracy: 0.9135


In [20]:
for pred, true in zip(test_pred, y_test[0:10]):
    print('{} - {:.2f}'.format(true, pred[0]))

1 - 0.36
0 - 0.20
0 - 0.64
1 - 0.98
0 - 0.00
0 - 0.00
1 - 0.97
0 - 0.00
1 - 1.00
1 - 1.00


### Add early stopping and regularization

In [21]:
# Input layer
inputs = tf.keras.Input(shape=(X_train.shape[1]))

# Hidden layer with regularization and ReLU
hidden = tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs)
hidden = tf.keras.activations.relu(hidden)

# Output layer with regularization and sigmoid
outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.001))(hidden)
outputs = tf.keras.activations.sigmoid(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

model.compile(
        optimizer=tf.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()],
)

model.summary()

Model: "RegularizedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_10 (Dense)            (None, 10)                90        
                                                                 
 tf.nn.relu_1 (TFOpLambda)   (None, 10)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
 tf.math.sigmoid_1 (TFOpLamb  (None, 1)                0         
 da)                                                             
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
____________________________________________________

In [22]:
epochs = 200

early_call = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_dev, y_dev),
          callbacks=[early_call])

Epoch 1/200
400/400 [==============================] - 2s 3ms/step - loss: 0.5382 - auc_1: 0.8246 - binary_accuracy: 0.7381 - val_loss: 0.4406 - val_auc_1: 0.8964 - val_binary_accuracy: 0.8034
Epoch 2/200
400/400 [==============================] - 1s 2ms/step - loss: 0.4196 - auc_1: 0.9041 - binary_accuracy: 0.8213 - val_loss: 0.3840 - val_auc_1: 0.9239 - val_binary_accuracy: 0.8472
Epoch 3/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3727 - auc_1: 0.9268 - binary_accuracy: 0.8528 - val_loss: 0.3435 - val_auc_1: 0.9389 - val_binary_accuracy: 0.8753
Epoch 4/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3358 - auc_1: 0.9415 - binary_accuracy: 0.8723 - val_loss: 0.3166 - val_auc_1: 0.9489 - val_binary_accuracy: 0.8869
Epoch 5/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3125 - auc_1: 0.9499 - binary_accuracy: 0.8848 - val_loss: 0.3019 - val_auc_1: 0.9539 - val_binary_accuracy: 0.8894
Epoch 6/200
400/400 [==============

### Gridsearch and Tensorboard
Run gridsearch over hidden layer size, L2 regularization, activation, check the outputs in Tensorboard

I recommend not to run Tensorboard from Jupyter notebook but from terminal directly

use "tensorboard --logdir logs" in command line

In [23]:
# hidden_sizes = [2, 5, 10, 20, 50]
# l2_regs = [0.01, 0.001, 0.0001]
# activations = ['relu', 'tanh']

hidden_sizes = [2, 5]
l2_regs = [0.01]
activations = ['relu', 'tanh']

epochs = 10
batch_size = 64

early_call = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', patience=10, restore_best_weights=True)

for activation in activations:
    for l2_reg in l2_regs:
        for hidden_size in hidden_sizes:
            if activation == 'relu':
                activate = tf.keras.activations.relu
            elif activation == 'tanh':
                activate = tf.keras.activations.tanh

            # Create Tensorboard Callback
            param_string = 'act-{},l2-{},hs-{}'.format(activation, l2_reg, hidden_size)
            log_dir = 'logs/binary_classification_test/' + param_string
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            # Input layer
            inputs = tf.keras.Input(shape=(X_train.shape[1]))

            # Hidden layer with regularization and ReLU
            hidden = tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
            hidden = activate(hidden)

            # Output layer with regularization and sigmoid
            outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(hidden)
            outputs = tf.keras.activations.sigmoid(outputs)

            model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

            model.compile(
                    optimizer=tf.optimizers.Adam(),
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy()],
            )

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, y_dev),
                      callbacks=[early_call, tensorboard_callback])

Epoch 1/10
200/200 [==============================] - 2s 4ms/step - loss: 1.0569 - AUC: 0.7086 - binary_accuracy: 0.5770 - val_loss: 0.8187 - val_AUC: 0.7528 - val_binary_accuracy: 0.6256
Epoch 2/10
200/200 [==============================] - 0s 2ms/step - loss: 0.7320 - AUC: 0.7577 - binary_accuracy: 0.6484 - val_loss: 0.6483 - val_AUC: 0.7970 - val_binary_accuracy: 0.7072
Epoch 3/10
200/200 [==============================] - 0s 2ms/step - loss: 0.6219 - AUC: 0.8001 - binary_accuracy: 0.7216 - val_loss: 0.5857 - val_AUC: 0.8277 - val_binary_accuracy: 0.7534
Epoch 4/10
200/200 [==============================] - 0s 2ms/step - loss: 0.5739 - AUC: 0.8227 - binary_accuracy: 0.7573 - val_loss: 0.5507 - val_AUC: 0.8412 - val_binary_accuracy: 0.7763
Epoch 5/10
200/200 [==============================] - 0s 2ms/step - loss: 0.5462 - AUC: 0.8340 - binary_accuracy: 0.7774 - val_loss: 0.5307 - val_AUC: 0.8445 - val_binary_accuracy: 0.7966
Epoch 6/10
200/200 [==============================] - 0s 2ms

In [24]:
%load_ext tensorboard
%tensorboard --logdir=logs/binary_classification_test